In [ ]:
%matplotlib inline

import numpy as np
import pickle
import json
from argparse import Namespace
import plotly.express as px
import matplotlib.pyplot as plt
import time
import yaml

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import networkx as nx
from pyvis.network import Network

from sklearn.decomposition import PCA
import phate
from umap import UMAP
from sklearn.manifold import TSNE

# Load simulated data

In [ ]:
"""load saved trajectories data for npz file
"""
# SEQ = "PT4"
SEQ = "PT4_hairpin"

# multiple trajectories
fnpz_data = "./data/pretraining/pretraining_{}.npz".format(SEQ)

data_npz = np.load(fnpz_data)

# asssign data to variables
for var in data_npz.files:
     locals()[var] = data_npz[var]

# recover full data based on coord_id, indices, and unique data
SIMS_adj = SIMS_adj_uniq[coord_id_S]
SIMS_scar = SIMS_scar_uniq[coord_id_S]
SIMS_G = SIMS_G_uniq[coord_id_S]
SIMS_pair = SIMS_pair_uniq[coord_id_S]

# print(SIMS_T.shape,SIMS_HT.shape,SIMS_HT_uniq.shape)
# print(SIMS_adj.shape,SIMS_scar.shape,SIMS_G.shape,SIMS_HT.shape,SIMS_pair.shape)
# print(SIMS_adj_uniq.shape,SIMS_scar_uniq.shape,SIMS_G_uniq.shape,SIMS_pair_uniq.shape) 
# print(SIMS_dict.shape,SIMS_dict_uniq.shape)
# print(coord_id_S.shape,indices_S.shape,trj_id.shape,data_embed.shape,occ_density_S.shape)
# print(pca_coords.shape,pca_all_coords.shape)
# print(phate_coords.shape,phate_all_coords.shape)
# print(umap_coord_2d.shape,umap_all_coord_2d.shape,umap_coord_3d.shape,umap_all_coord_3d.shape)
# print(tsne_coord_2d.shape,tsne_all_coord_2d.shape,tsne_coord_3d.shape,tsne_all_coord_3d.shape)

# ViDa Model

### Make dataset

In [ ]:
data_tup = (torch.Tensor(SIMS_scar_uniq),
            torch.Tensor(SIMS_G_uniq),
            )
data_dataset = torch.utils.data.TensorDataset(*data_tup)

In [ ]:
# split the dataset into train and validation
data_size = len(data_dataset)
train_size = int(0.7 * data_size)
val_size = data_size - train_size

train_data, val_data = torch.utils.data.random_split(data_dataset, [train_size, val_size], 
                                                     generator=torch.Generator().manual_seed(42))

print(data_size, len(train_data), len(val_data))

### Set up configurations

In [ ]:
# set up hyperparameters
input_dim = data_tup[0].shape[-1]

config = Namespace(
    seq = SEQ,
    type = 'vida-gsae',
    device = 'mps', # change to cuda if using GPU
    batch_size = 256,
    input_dim = input_dim,
    output_dim = input_dim,
    latent_dim = 25, # bottleneck dimension
    hidden_dim = 400,
    n_epochs = 60, # 60 for PT4_hairpin, PT0, 100 for others
    learning_rate = 0.0001, # learning rate
    log_interval = 10, # how many batches to wait before logging training status    
    
    # hyperparameters for loss function
    alpha = 1.0,
    beta = 1e-5,
    gamma = 1.0
)

### Make datalaoder

In [ ]:
data_loader = torch.utils.data.DataLoader(data_dataset, batch_size=config.batch_size,
                                          shuffle=False)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=config.batch_size, 
                                           shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=config.batch_size,
                                         shuffle=False)

In [ ]:
print('train_loader: ', len(train_loader.dataset), len(train_loader), train_loader.batch_size)
print('val_loader: ', len(val_loader.dataset), len(val_loader), val_loader.batch_size)

### Encoder

In [ ]:
class Encoder(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, latent_dim):
        '''
        Args:
        ----
            - input_dim: the dimension of the input node feature
            - hiddent_dim: the dimension of the hidden layer
            - latent_dim: the dimension of the latent space (bottleneck layer)
        '''
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        
        self.fc1 = nn.Linear(self.input_dim, self.hidden_dim)
        self.bn1 = nn.BatchNorm1d(self.hidden_dim)
        self.fc2 = nn.Linear(400, 400)
        self.bn2 = nn.BatchNorm1d(400)
        
        # Split the result into mu and var components
        # of the latent Gaussian distribution, note how we only output
        # diagonal values of covariance matrix. Here we assume
        # they are conditionally independent
        self.hid2mu = nn.Linear(400, self.latent_dim)
        self.hid2logvar = nn.Linear(400, self.latent_dim)
        
    def forward(self, x):
        x = self.bn1(F.relu(self.fc1(x)))
        x = self.bn2(F.relu(self.fc2(x)))
        mu = self.hid2mu(x)
        logvar = self.hid2logvar(x)
        return mu, logvar


### Decoder

In [ ]:
class Decoder(nn.Module):
    
    def __init__(self, latent_dim, hidden_dim, output_dim):
        '''
        Args:
        ----
            - latent_dim: the dimension of the latent space (bottleneck layer)
            - hiddent_dim: the dimension of the hidden layer
            - output_dim: the dimension of the output node feature
        '''
        super(Decoder, self).__init__()
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.fc1 = nn.Linear(self.latent_dim, 400)
        # self.bn1 = nn.BatchNorm1d(self.hidden_dim)
        # self.fc2 = nn.Linear(400, 400)
        # self.bn2 = nn.BatchNorm1d(400)
        self.fc3 = nn.Linear(400, self.output_dim)
        
    def forward(self, z):
        x = F.relu(self.fc1(z))
        x = self.fc3(x)
        return x

### Regressor

In [ ]:
class Regressor(nn.Module):
    
    def __init__(self, latent_dim):
        '''
        The regressor is used to predict the energy of the node
        
        Args:
        ----
            - latent_dim: the dimension of the latent space (bottleneck layer)
        '''
        super(Regressor, self).__init__()
        self.latent_dim = latent_dim
        
        self.regfc1 = nn.Linear(self.latent_dim, 15)
        self.regfc2 = nn.Linear(15, 1)
        
    def forward(self, z):
        y = F.relu(self.regfc1(z))
        y = self.regfc2(y)
        return y

### VIDA model

In [ ]:
class VIDA(nn.Module):
    
    def __init__(self, encoder, decoder, regressor):
        '''
        Args:
        ----
            - input_dim: the dimension of the input node feature
            - hiddent_dim: the dimension of the hidden layer
            - latent_dim: the dimension of the latent space (bottleneck layer)
            - output_dim: the dimension of the output node feature (same as input_dim)
        '''
        super(VIDA, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.regressor = regressor
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = mu + eps*std
        return z
        
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        y_pred = self.regressor(z)
        return x_recon, y_pred, z, mu, logvar

### Loss functions

In [ ]:
def vae_loss(x_recon, x, mu, logvar):
    '''
    Compute the VAE loss
    
    Args:
        - x_recon: the reconstructed node feature
        - x: the original node feature
        - mu: the mean of the latent space
        - logvar: the log variance of the latent space
    
    Returns:
    - loss: PyTorch Tensor containing (scalar) the loss for the VAE
    '''
    BCE = F.mse_loss(x_recon.flatten(), x.flatten())
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # return BCE + KLD
    return BCE, KLD


def pred_loss(y_pred, y):
    '''
    Compute the energy prediction loss
    
    Args:
    ----
        - y_pred: the predicted energy of the node
        - y: the true energy of the node
    
    Returns:
        - loss: PyTorch Tensor containing (scalar) the loss for the prediction
    '''
    return F.mse_loss(y_pred.flatten(), y.flatten())

### Train VIDA

In [ ]:
# validation function
def validate(config, model, val_loader, vae_loss, pred_loss):
    model.to(config.device)
    model.eval()
    
    val_loss = []; val_bce = []; val_kld = []; val_pred = []
    
    # Disable gradient calculation to speed up inference
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(val_loader):
            
            # Configure input
            x = x.to(config.device)
            y = y.to(config.device)
            
            # embedding
            x_recon, y_pred, z, mu, logvar = model(x)
            
            recon_loss, kl_loss = vae_loss(x_recon, x, mu, logvar)
            
            p_loss = pred_loss(y_pred, y)
            
            # scaling the loss
            recon_loss = config.alpha * recon_loss
            kl_loss = config.beta * kl_loss
            p_loss = config.gamma * p_loss
            
            loss = recon_loss + kl_loss + p_loss # total loss
            
            val_loss.append(loss.item())
            val_bce.append(recon_loss.item())
            val_kld.append(kl_loss.item())
            val_pred.append(p_loss.item())
            
    print('Validation Loss: {:.4f}'.format(np.mean(val_loss)))
    
    return np.mean(val_loss), np.mean(val_bce), np.mean(val_kld), np.mean(val_pred)
            

In [ ]:
def train(config, model, train_loader, val_loader,
          optimizer, vae_loss, pred_loss,
          ):
    '''
    Train VIDA!
    
    Args:
    ----
        - config: Experiment configurations
        - model: Pytorch VIDA model
        - train_loader: Pytorch DataLoader for training set
        - val_loader: Pytorch DataLoader for validation set
        - optimizer: Pytorch optimizer
        - vae_loss: the VAE loss function
        - pred_loss: the energy prediction loss function
    '''
    
    model.to(config.device)
    model.train()
    
    log_dir = f'./model_config/{time.strftime("%m%d-%H%M")}'
    writer = SummaryWriter(log_dir=log_dir)
    
    # save the config file
    with open(f'{log_dir}/hparams.yaml','w') as f:
        yaml.dump(config,f)
            
    print('\n ------- Start Training -------')
    for epoch in range(config.n_epochs):
        
        training_loss = []
        
        for batch_idx, (x, y) in enumerate(train_loader):  # mini batch
            
            # Configure input
            x = x.to(config.device)
            y = y.to(config.device)
            
            # ------------------------------------------
            #  Train VIDA
            # ------------------------------------------
            optimizer.zero_grad()
            
            # get the reconstructed nodes, predicted energy, and the embeddings
            x_recon, y_pred, z, mu, logvar = model(x)
 
            # compute the total loss
            recon_loss, kl_loss = vae_loss(x_recon, x, mu, logvar)
            p_loss = pred_loss(y_pred, y)
                        
            # scaling the loss
            recon_loss = config.alpha * recon_loss
            kl_loss = config.beta * kl_loss
            p_loss = config.gamma * p_loss
    
            # total loss
            loss = recon_loss + kl_loss + p_loss 
            
            training_loss.append(loss.item())
            
            # backpropagation and optimization
            loss.backward()
            optimizer.step()
            
            # ------------------------------------------
            # Log Progress
            # ------------------------------------------
            if batch_idx % config.log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(x), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.item()))
                
                writer.add_scalar('training loss',
                                  loss.item(),
                                  epoch * len(train_loader) + batch_idx)
                writer.add_scalar('recon loss',
                                  recon_loss.item(),
                                  epoch * len(train_loader) + batch_idx)
                writer.add_scalar('kl loss',
                                  kl_loss.item(),
                                  epoch * len(train_loader) + batch_idx)
                writer.add_scalar('pred loss',
                                  p_loss.item(),
                                  epoch * len(train_loader) + batch_idx)
                    
        print ('====> Epoch: {} Average loss: {:.4f}'.format(epoch, np.mean(training_loss)))
        writer.add_scalar('epoch training loss', np.mean(training_loss), epoch)
        
        # validation
        val_loss, val_bce, val_kld, val_pred = validate(config, model, val_loader, vae_loss, pred_loss)
        writer.add_scalar('validation loss', val_loss, epoch)
        writer.add_scalar('val_recon loss', val_bce, epoch)
        writer.add_scalar('val_kl loss', val_kld, epoch)
        writer.add_scalar('val_pred loss', val_pred, epoch)        
    
    writer.close()  
    print('\n ------- Finished Training -------')
    
    # save the model
    torch.save(model.state_dict(), f'{log_dir}/model.pt')
    

In [ ]:
# define models
encoder = Encoder(input_dim=config.input_dim, hidden_dim=config.hidden_dim, latent_dim=config.latent_dim)
decoder = Decoder(latent_dim=config.latent_dim, hidden_dim=config.hidden_dim, output_dim=config.output_dim)
regressor = Regressor(latent_dim=config.latent_dim)

vida = VIDA(encoder, decoder, regressor)

# define optimizer
optimizer = torch.optim.Adam(vida.parameters(), lr=config.learning_rate)

In [ ]:
# train VIDA
train(config, vida, train_loader, val_loader, optimizer, vae_loss, pred_loss)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir model_config/ --host localhost --port 8000
#  http://localhost:8000

### Load trained model

In [ ]:
# model = VIDA(encoder, decoder, regressor)
# model.load_state_dict(torch.load('./model_config/0223-1633/model.pt'))

### Get embeddings

In [ ]:
model = VIDA(encoder, decoder, regressor)

In [ ]:
# do inference
model.to(config.device).eval()

with torch.no_grad():
        _, _, z, _, _ = model(data_loader.dataset.tensors[0].to(config.device))        

In [ ]:
data_embed = z.to('cpu').numpy()
data_embed.shape

In [ ]:
print(data_embed.max(), data_embed.min(), data_embed.mean(), data_embed.std())

### PCA

In [ ]:
# # do PCA for GSAE embeded data
pca_coords = PCA(n_components=3).fit_transform(data_embed)

# # get all pca embedded states coordinates
pca_all_coords = pca_coords[coord_id_S]  # multiple trj

pca_coords.shape, pca_all_coords.shape

In [ ]:
(np.unique(pca_coords,axis=0)).shape, (np.unique(pca_all_coords,axis=0)).shape

### PHATE

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_embed = scaler.fit_transform(data_embed)
data_embed

In [ ]:
# # do PHATE for GSAE embeded data
phate_operator = phate.PHATE(n_jobs=-2)
phate_coords = phate_operator.fit_transform(data_embed)

# # get all phate embedded states coordinates
phate_all_coords = phate_coords[coord_id_S]

phate_coords.shape, phate_all_coords.shape

In [ ]:
phate_coords.shape, (np.unique(phate_coords,axis=0)).shape, (np.unique(phate_all_coords,axis=0)).shape

### UMAP

In [ ]:
# UMAP set
umap_2d = UMAP(n_components=2, init='random', random_state=0)
# umap_3d = UMAP(n_components=3, init='random', random_state=0)

# UMAP 2D fit tranform
umap_coord_2d = umap_2d.fit_transform(data_embed)
umap_all_coord_2d = umap_coord_2d[coord_id_S]  

# UMAP 3D fit tranform
# umap_coord_3d = umap_3d.fit_transform(data_embed)
# umap_all_coord_3d = umap_coord_3d[coord_id_S]

print((np.unique(umap_coord_2d,axis=0)).shape, (np.unique(umap_coord_3d,axis=0)).shape)
print(umap_all_coord_2d.shape, (np.unique(umap_all_coord_2d,axis=0)).shape)
# print(umap_all_coord_3d.shape, (np.unique(umap_all_coord_3d,axis=0)).shape)

### Save DRs

In [ ]:
""" Save all DRs
"""
# save for python
fnpz_data = f"data/vida_data/new_embed/{SEQ}_{config.n_epochs}epoch.npz"
with open(fnpz_data, 'wb') as f:
    np.savez(f,
            # SIMS data
            SIMS_T=SIMS_T, SIMS_HT=SIMS_HT, SIMS_HT_uniq=np.array(avg_time, dtype=np.float64),
            SIMS_adj_uniq=SIMS_adj_uniq, SIMS_scar_uniq=SIMS_scar_uniq,
            SIMS_G_uniq=SIMS_G_uniq, SIMS_pair_uniq=SIMS_pair_uniq,
            SIMS_dict=SIMS_dict, SIMS_dict_uniq=SIMS_dict_uniq,
            # Indices
            coord_id_S=coord_id_S, indices_S=indices_S,trj_id=trj_id,
            # embed data and occpancy density
            data_embed=data_embed, occ_density_S=occ_density_S,
            # plotting data
            pca_coords=pca_coords, pca_all_coords=pca_all_coords,
            phate_coords=phate_coords, phate_all_coords=phate_all_coords,
            umap_coord_2d=umap_coord_2d, umap_all_coord_2d=umap_all_coord_2d,
            )

# Visualize

In [ ]:
SEQ

### PCA Vis

In [ ]:
%%script false --no-raise-error

%matplotlib inline
X = pca_all_coords[:,0]
Y = pca_all_coords[:,1]
Z = pca_all_coords[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G,
          cmap='plasma',
          s=20
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]*0.95),fontsize=15,c="yellow", horizontalalignment='center')

In [ ]:
%matplotlib inline
X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)


annotations=["I","F"]
x = [X[0],X[all_nodes[-1]]]
y = [Y[0],Y[all_nodes[-1]]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="black")

In [ ]:
%%script false --no-raise-error

X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]

# PCA: 3 components
fig,ax = plt.subplots(figsize=(8,6))
ax = plt.axes(projection ="3d")

im = ax.scatter3D(X,Y,Z,
          c=SIMS_G_uniq,      
          cmap='plasma')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
z = [Z[0], Z[-1]]
ax.scatter(x,y,z,s=100,c="green",alpha=1)

In [ ]:
%%script false --no-raise-error

X = pca_coords[:,0]
Y = pca_coords[:,1]
Z = pca_coords[:,2]


# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y,
          c=SIMS_pair_uniq,
          cmap='plasma',
          s=15
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="yellow")

#### Try use PCA directly without AE

In [ ]:
pca_coords1 = PCA(n_components=3).fit_transform(SIMS_scar_uniq)   # multiple trj

X = pca_coords1[:,0]
Y = pca_coords1[:,1]
Z = pca_coords1[:,2]

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[-1]]
y = [Y[0],Y[-1]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="black")

In [ ]:
cm = PCA(n_components=25)
cm.fit(data_embed)

PC_values = np.arange(cm.n_components_) + 1
plt.plot(PC_values, np.cumsum(cm.explained_variance_ratio_), 'ro-', linewidth=2)
plt.title('Scree Plot: PCA')
plt.xlabel('Number of principal components')
plt.ylabel('Cumulative explained variance');
# plt.xticks(np.arange(0, data_embed.shape[-1]+1, 1))

plt.show()

In [ ]:
np.cumsum(cm.explained_variance_ratio_)

### PHATE Vis

In [ ]:
%%script false --no-raise-error

X_phate = phate_all_coords[:,0]
Y_phate = phate_all_coords[:,1]

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X_phate,Y_phate,
                c=SIMS_G,   # multiple trj               
                cmap='plasma',
               )

plt.colorbar(im)

annotations=["I","F"]
x = [X_phate[0],X_phate[-1]]
y = [Y_phate[0],Y_phate[-1]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=30,c="black")

In [ ]:
X_phate = phate_coords[:,0]
Y_phate = phate_coords[:,1]

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X_phate,Y_phate,
                c=SIMS_G_uniq,            
                cmap='plasma',
               )

plt.colorbar(im)

annotations=["I","F"]
x = [X_phate[0],X_phate[all_nodes[-1]]]
y = [Y_phate[0],Y_phate[all_nodes[-1]]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=30,c="black")

#### PHATE without AE

In [ ]:
phate_operator = phate.PHATE(n_jobs=-2)
phate1 = phate_operator.fit_transform(SIMS_scar_uniq)   # multiple trj

fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(phate1[:,0],
          phate1[:,1],
          c=SIMS_G_uniq, 
          cmap='plasma',
        )

plt.colorbar(im)

annotations=["I","F"]
x = [phate1[:,0][0],phate1[:,0][-1]]
y = [phate1[:,1][0],phate1[:,1][-1]]
plt.scatter(x,y,s=50, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i],y[i]),fontsize=20,c="black")

### UMAP Vis

In [ ]:
X = umap_coord_2d[:,0]
Y = umap_coord_2d[:,1]
cmap = plt.cm.plasma
cmap_r = plt.cm.get_cmap('plasma_r')

# PCA: 2 components
fig,ax = plt.subplots(figsize=(8,6))
im = ax.scatter(X, Y, 
          c = SIMS_G_uniq,
          cmap=cmap,
          s=10
        )
 
plt.colorbar(im)

annotations=["I","F"]
x = [X[0],X[all_nodes[-1]]]
y = [Y[0],Y[all_nodes[-1]]]
plt.scatter(x,y,s=150, c="green", alpha=1)
for i, label in enumerate(annotations):
    plt.annotate(label, (x[i]-0.3,y[i]-0.3),fontsize=15,c="yellow")

In [ ]:
%%script false --no-raise-error

# directly UMAP 2D
umap_coord_2dscar = umap_2d.fit_transform(SIMS_scar_uniq)

fig_2d = px.scatter(
    umap_coord_2dscar, x=0, y=1,color=SIMS_G_uniq
)
fig_2d.update_traces(marker_size=3)
fig_2d.show()



In [ ]:
%%script false --no-raise-error

fig_2d = px.scatter(
    umap_coord_2d, x=0, y=1,color=SIMS_G_uniq
)
fig_2d.update_traces(marker_size=3)


fig_3d = px.scatter_3d(
    umap_coord_3d, x=0, y=1, z=2,color=SIMS_G_uniq
)

fig_3d.update_traces(marker_size=2)

fig_2d.show()
fig_3d.show()

